In [142]:
#Liên kết với google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [161]:
#Thêm vào các thư viện cần thiết
import pandas as pd
import os
from nltk.translate.bleu_score import sentence_bleu
import warnings
import gensim
from scipy.spatial import distance
import numpy as np

In [144]:
#Đường dẫn
os.getcwd()
os.chdir('/content/drive/My Drive/Data/Final/')

In [145]:
#Đọc file
df1 = pd.read_csv('chatbot data.csv')
print(df1.shape)
df1.head()

(1480, 3)


,Unnamed: 0,user_a,user_b
0,0,bạn tên gì?,Mình tên Trường <eos> còn bạn tên gì?
1,1,Tui tên Nam <eos> Bạn học ở đâu?,tui học ở TDTU nè <eos> còn bạn học ở trường n...
2,2,tui học ở TDTU luôn <eos> Bạn học ngành gì,mình học Công Nghệ Thông Tin
3,3,Chuyên ngành gì vậy bạn ?,mình học bên Kỹ Thuật Phần Mềm á <eos> còn bạn...
4,4,mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...,Mình là người Đồng tháp <eos> bạn biết Đồng Th...


In [146]:
df2 = pd.read_csv('dữ liệu chatbot question-answer short style.csv')
print(df2.shape)
df2.head()

(5900, 3)


,Unnamed: 0,user_a,user_b
0,0,Thích mẫu người nào,"Dễ thương, tóc dài, da trắng"
1,1,Có crush ai không,Có 1 bạn cùng lớp
2,2,Tại sao lại thích bạn dó,Vì đáp ứng những yêu cầu của tao
3,3,Có hay nói chuyện không,Hay nhắn tin messenger
4,4,Bạn kia có bắt chuyện trước không,Có đôi khi


In [147]:
#Gộp 2 dataframe lại
frames = [df1, df2]

dataframe = pd.concat(frames)
print(dataframe.shape)
dataframe = dataframe.dropna().reset_index(drop = True)
dataframe.head()

(7380, 3)


,Unnamed: 0,user_a,user_b
0,0,bạn tên gì?,Mình tên Trường <eos> còn bạn tên gì?
1,1,Tui tên Nam <eos> Bạn học ở đâu?,tui học ở TDTU nè <eos> còn bạn học ở trường n...
2,2,tui học ở TDTU luôn <eos> Bạn học ngành gì,mình học Công Nghệ Thông Tin
3,3,Chuyên ngành gì vậy bạn ?,mình học bên Kỹ Thuật Phần Mềm á <eos> còn bạn...
4,4,mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...,Mình là người Đồng tháp <eos> bạn biết Đồng Th...


##**Bleu Score**

In [148]:
#Tập dữ liệu câu hỏi
reference = []
for sen in dataframe['user_a'].values:
  sent = []
  #Loại bỏ những ký tự không phải là chữ cái + bỏ in hoa
  for word in sen.strip().split(' '):
    tmp = ''.join(x for x in word if x.isalpha())
    if tmp == '' or tmp == 'eos':
      continue
    else:
      sent.append(tmp.lower())
  reference.append(sent)

reference[:5]

[['bạn', 'tên', 'gì'],
 ['tui', 'tên', 'nam', 'bạn', 'học', 'ở', 'đâu'],
 ['tui', 'học', 'ở', 'tdtu', 'luôn', 'bạn', 'học', 'ngành', 'gì'],
 ['chuyên', 'ngành', 'gì', 'vậy', 'bạn'],
 ['mình',
  'học',
  'quản',
  'trị',
  'kinh',
  'doanh',
  'bạn',
  'là',
  'người',
  'ở',
  'đâu',
  'vậy']]

In [149]:
#So câu được nhập vào với từng câu trong tập dữ liệu
#Câu nào có bleu score cao nhất thì lấy vị trí để lấy câu trả lời tương ứng
def bleu_sc(sentence, dic):
  candidate = []
  #Loại bỏ những ký tự không phải là chữ cái + bỏ in hoa
  for word in sentence.strip().split(' '):
    tmp = ''.join(x for x in word if x.isalpha())
    if tmp == '' or tmp == 'eos':
      continue
    else:
      candidate.append(tmp.lower())

  max_score = 0
  index = 0
  for i in range(len(dic)):
    #Bleu score với unigram
    score = sentence_bleu([dic[i]], candidate, (1, 0, 0, 0))
    if score > max_score:
      max_score = score
      index = i
  #Nếu câu nhập vào không khớp với câu nào trong tập dữ liệu -> trả về -1
  if max_score == 0:
    return -1
  return index

In [150]:
print('----------Bắt đầu cuộc hội thoại----------')
warnings.filterwarnings("ignore")
while True:
  x = input('Bạn:  ')
  #Dùng !end để kết thúc cuộc hội thoại
  if x == '!end':
    print('Bot:  Tạm biệt.')
    print('----------Cuộc hội thoại kết thúc----------')
    break

  index = bleu_sc(x, reference)
  if index == -1:
    print('Bot:  Không rõ câu nói. Vui lòng nhập lại.')
    continue
  #In ra câu trả lời
  print('Bot: ',dataframe.iloc[index][2])

----------Bắt đầu cuộc hội thoại----------
Bạn:  !end
Bot:  Tạm biệt.
----------Cuộc hội thoại kết thúc----------


##**Word2Vec**

In [151]:
#Tập dữ liệu câu hỏi
reference_2 = []
for sen in df1['user_a'].values:
  sent = []
  #Loại bỏ những ký tự không phải là chữ cái + bỏ in hoa
  for word in sen.strip().split(' '):
    tmp = ''.join(x for x in word if x.isalpha())
    if tmp == '' or tmp == 'eos':
      continue
    else:
      sent.append(tmp.lower())
  reference_2.append(sent)

reference_2[:5]

[['bạn', 'tên', 'gì'],
 ['tui', 'tên', 'nam', 'bạn', 'học', 'ở', 'đâu'],
 ['tui', 'học', 'ở', 'tdtu', 'luôn', 'bạn', 'học', 'ngành', 'gì'],
 ['chuyên', 'ngành', 'gì', 'vậy', 'bạn'],
 ['mình',
  'học',
  'quản',
  'trị',
  'kinh',
  'doanh',
  'bạn',
  'là',
  'người',
  'ở',
  'đâu',
  'vậy']]

In [175]:
#Tạo và train model Word2Vec
model = gensim.models.Word2Vec(min_count = 1, window = 2)
model.build_vocab(reference_2, progress_per=10000)
model.train(reference_2, total_examples = model.corpus_count, epochs = 100)
model['xin']

array([-8.82235318e-02,  2.32475787e-01, -9.55088586e-02, -2.36876067e-02,
        4.07829173e-02, -5.53072214e-01, -2.48605430e-01,  9.25661981e-01,
        2.89369076e-01, -6.02485359e-01, -3.38015296e-02,  2.69898087e-01,
        2.76810557e-01,  2.84593314e-01, -4.58531499e-01, -5.28473258e-01,
        5.96180916e-01,  1.11258626e-01,  3.22058171e-01,  1.66720077e-01,
       -6.68020010e-01, -5.29352129e-01, -2.25632757e-01, -7.61565983e-01,
        2.31486097e-01, -7.42462650e-02,  6.89306438e-01, -2.11895153e-01,
       -2.55560845e-01,  7.00066864e-01,  7.31419548e-02,  5.93245700e-02,
        2.24772200e-01, -2.61346791e-02,  2.19917875e-02,  2.65134554e-02,
        4.66888845e-01, -5.19421458e-01,  2.38633156e-01, -1.59119979e-01,
       -7.81990945e-01, -4.78848591e-02,  2.98474371e-01,  1.72863364e-01,
       -3.85719091e-01,  4.72910792e-01,  5.92039526e-02, -6.86996400e-01,
       -1.85761705e-01, -4.14485097e-01,  2.31464684e-01, -5.96558452e-01,
        7.16085553e-01, -

In [176]:
#Tạo vector cho từng câu hỏi trong tập dữ liệu
#Lấy trung bình cộng của từng từ trong câu
vector_sentence = []
for sen in reference_2:
  tmp = [0]
  for word in sen:
    tmp = tmp + model[word]
  tmp = tmp / len(sen)
  vector_sentence.append(tmp)

vector_sentence[0]

array([-0.26762037,  0.16558467,  0.21478659, -0.15977307, -0.14104253,
       -0.63594747, -0.15906704,  0.71259815,  0.62531514, -0.40935704,
       -0.06065054,  0.47352229,  0.80656171, -0.1331811 ,  0.04563736,
       -0.67792513,  0.55659633, -0.10618339,  0.15773895,  0.44286661,
       -0.49975656, -0.14356362, -0.2036985 , -0.5082672 ,  0.34068413,
       -0.45213164, -0.26768086,  0.08932362, -0.01695632,  0.41597133,
       -0.36907131,  0.10193053, -0.26611165, -0.06405945,  0.14998808,
        0.48218783,  0.13162559,  0.01226608,  0.62018095,  0.29281131,
       -0.58272963,  0.14723849,  0.36305417, -0.11586804, -0.6815889 ,
        0.63827721, -0.2191115 , -0.45163247,  0.24087476, -0.05448198,
        0.44747763, -0.12249587,  0.0579732 , -0.78407363, -0.29642848,
       -0.01154617,  0.33804319, -1.0702889 ,  0.4935158 ,  0.01855656,
       -0.31937775,  0.33487896, -0.41732588,  1.10145026, -0.63793457,
       -0.48952213, -0.17655443,  0.48484253,  0.17698329,  0.64

In [177]:
def euclid_dist(sentence, dic):
  word_list = []
  #Loại bỏ những ký tự không phải là chữ cái + bỏ in hoa
  for word in sentence.strip().split(' '):
    tmp = ''.join(x for x in word if x.isalpha())
    if tmp == '' or tmp == 'eos':
      continue
    else:
      word_list.append(tmp.lower())

  vect = [0]
  for word in word_list:
    vect = vect + model[word]
  vect = vect / len(word_list)

  max_dist = 0
  index = 0
  for i in range(len(dic)):
    #Euclid
    dist = distance.euclidean(dic[i], vect)
    if dist > max_dist:
      max_dist = dist
      index = i
  #Nếu câu nhập vào không khớp với câu nào trong tập dữ liệu -> trả về -1
  if max_dist == 0:
    return -1
  
  return index

In [178]:
print('----------Bắt đầu cuộc hội thoại----------')
warnings.filterwarnings("ignore")
while True:
  x = input('Bạn:  ')
  #Dùng !end để kết thúc cuộc hội thoại
  if x == '!end':
    print('Bot:  Tạm biệt.')
    print('----------Cuộc hội thoại kết thúc----------')
    break

  index = euclid_dist(x, vector_sentence)
  if index == -1:
    print('Bot:  Không rõ câu nói. Vui lòng nhập lại.')
    continue
  #In ra câu trả lời
  print('Bot: ',dataframe.iloc[index][2])

----------Bắt đầu cuộc hội thoại----------
Bạn:  Xin chào
Bot:  okay 
Bạn:  ĐI chơi không?
Bot:  mình thường đá bóng và bơi lội.
Bạn:  Hôm qua thi được mấy điểm?
Bot:  mình thường đá bóng và bơi lội.
Bạn:  Bot ngu
Bot:  mình thường đá bóng và bơi lội.
Bạn:  !end
Bot:  Tạm biệt.
----------Cuộc hội thoại kết thúc----------
